In [1]:
# 절대 임포트 설정
from __future__ import division, print_function, absolute_import

# 필요한 라이브러리들을 임포트
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data

%matplotlib inline

In [2]:
"""
define all the constants
"""
numEpochs = 200
seriesLength = 500000
backpropagationLength = 250
stateSize = 4
numClasses = 10
echoStep = 3
batchSize = 250
num_batches = seriesLength // batchSize // backpropagationLength
print(num_batches)
param_sparsity = 0.01
param_weight_decay = 0.0001
param_sparse_panelty = 3

learning_rate_Gradient_Descent = 0.5
softmax_classifier_iterations = 1000 # Softmax Classifier iteration 횟수 
  
display_step = 1        # 몇 Step마다 log를 출력할지 결정한다.
examples_to_show = 10   # reconstruct된 이미지 중 몇개를 보여줄지를 결정한다. 

8


In [3]:
"""
generate data
"""
def generateData():
    #x = np.array(np.random.choice(25, seriesLength, p=[0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04]))
    x = np.random.random((5, 50000))
    
    y = np.roll(x, echoStep)
    y[0:echoStep] = 0
    
    print("x :", x.shape)
    print("y :", y.shape)
    print("batchSize : ", batchSize)

    x = x.reshape((batchSize, -1))
    y = y.reshape((batchSize, -1))
    
    print("x_reshape :", x.shape)
    print("y_reshape :", y.shape)

    return (x, y)

In [4]:
def regularization(weights):
    return tf.nn.l2_loss(weights)

def Single_Layer_Autoencoder(Input_Layer, n_input, n_hidden):
    with tf.name_scope("Hidden_Layer") as scope:
        # 히든 레이어 1을 위한 Weights와 Biases
        W_ih = tf.Variable(tf.random_normal([n_input, n_hidden]), name = "Weight_IH")  # IH = Input_Hidden
        b_ih = tf.Variable(tf.random_normal([n_hidden]), name = "Bias_IH")
        H_Layer = tf.nn.sigmoid(tf.matmul(Input_Layer, W_ih) + b_ih, name = "Hidden")     # 히든레이어 1의 activation (sigmoid 함수를 사용)
               
    with tf.name_scope("Reconstructed_Layer") as scope:
        # Output 레이어를 위한 Weights와 Biases
        W_hr = tf.Variable(tf.random_normal([n_hidden, n_input]), name = "Weight_HR")  # HR = Hidden_Reconstructed
        b_hr = tf.Variable(tf.random_normal([n_input]), name = "Bias_HR")
        X_reconstructed = tf.nn.sigmoid(tf.matmul(H_Layer,W_hr) + b_hr, name = "Hidden_HR")   # Output 레이어의 activation (sigmoid 함수를 사용)
    
    with tf.name_scope("cost") as scope:        
    # Autoencoder Optimization을 위한 파라미터들
        average_act_hidden = tf.reduce_mean(H_Layer,axis=0)   #Average hidden layer over all data points in X, Page 14 in https://web.stanford.edu/class/cs294a/sparseAutoencoder_2011new.pdf
        diff = Input_Layer - X_reconstructed
        KL = Kullback_Leibler_divergence(rho, average_act_hidden)
        cost= 0.5*tf.reduce_mean(tf.reduce_sum(tf.pow(diff,2),axis=1)) + 0.5*Lambda*(tf.nn.l2_loss(W_ih) + tf.nn.l2_loss(W_hr)) + Beta*tf.reduce_sum(KL)
        cost_summary = tf.summary.scalar("cost", cost)
        

    return H_Layer, X_reconstructed, cost

def Kullback_Leibler_divergence(rho, rho_hat):
    return rho * tf.log(rho) - rho * tf.log(rho_hat) + (1 - rho) * tf.log(1 - rho) - (1 - rho) * tf.log(1 - rho_hat)

In [5]:
rho = param_sparsity
Lambda = param_weight_decay
Beta = param_sparse_panelty

# 학습에 필요한 변수들 설정
with tf.name_scope("Tensor_initialize"):
       
    batchXHolder = tf.placeholder(tf.float32, [batchSize, backpropagationLength], name="x_input")
    batchYHolder = tf.placeholder(tf.int32, [batchSize, backpropagationLength], name="y_input")    
        
with tf.name_scope("Auto_Encoder_01"):
    H1, O1, cost1 = Single_Layer_Autoencoder(batchXHolder, batchSize, batchSize)
    optmzr1 = tf.train.GradientDescentOptimizer(learning_rate_Gradient_Descent).minimize(cost1)

with tf.name_scope("Auto_Encoder_02"):
    H2, O2, cost2 = Single_Layer_Autoencoder(H1, batchSize, batchSize)
    optmzr2 = tf.train.GradientDescentOptimizer(learning_rate_Gradient_Descent).minimize(cost2)

with tf.name_scope("Auto_Encoder_03"):
    H3, O3, cost3 = Single_Layer_Autoencoder(H2, batchSize, batchSize)
    optmzr3 = tf.train.GradientDescentOptimizer(learning_rate_Gradient_Descent).minimize(cost3)

with tf.name_scope("Auto_Encoder_04"):
    H4, O4, cost4 = Single_Layer_Autoencoder(H3, batchSize, batchSize)
    optmzr4 = tf.train.GradientDescentOptimizer(learning_rate_Gradient_Descent).minimize(cost4)

with tf.name_scope("Long_Short_Memory_Networks"):
    cellState = tf.placeholder(tf.float32, [batchSize, stateSize])
    hiddenState = tf.placeholder(tf.float32, [batchSize, stateSize])
    initState = rnn.LSTMStateTuple(cellState, hiddenState)
    
    W = tf.Variable(np.random.rand(stateSize+1, stateSize), dtype=tf.float32, name="weight1")
    bias1 = tf.Variable(np.zeros((1,stateSize)), dtype=tf.float32)
    W2 = tf.Variable(np.random.rand(stateSize, numClasses),dtype=tf.float32, name="weight2")
    bias2 = tf.Variable(np.zeros((1,numClasses)), dtype=tf.float32)
    
    tf.summary.histogram(name="LSTM_weights", values=W)
        
    # Unpack columns
    inputsSeries = tf.split(axis=1, num_or_size_splits=backpropagationLength, value=H4)
    labelsSeries = tf.unstack(batchYHolder, axis=1)
        
    # Forward passes
    cell = rnn.BasicLSTMCell(stateSize, state_is_tuple=True)
    statesSeries, currentState = rnn.static_rnn(cell, inputsSeries, initState)
    
    # calculate loss
    logits_series = [tf.matmul(state, W2) + bias2 for state in statesSeries]
    print("logits_series :",len(logits_series))
    predictions_series = [tf.nn.softmax(logits) for logits in logits_series]
    print("predictions_series :", len(predictions_series))
    losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits) for logits, labels in zip(logits_series,labelsSeries)]
    total_loss = tf.reduce_mean(losses, name="total_loss")
    train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss, name="training")

logits_series : 250
predictions_series : 250


In [ ]:
"""
plot computation
"""
def plot(loss_list, predictions_series, batchX, batchY):
    plt.subplot(2, 3, 1)
    plt.cla()
    plt.plot(loss_list)

    for batchSeriesIdx in range(5):
        oneHotOutputSeries = np.array(predictions_series)[:, batchSeriesIdx, :]
        singleOutputSeries = np.array([(1 if out[0] < 0.5 else 0) for out in oneHotOutputSeries])

        plt.subplot(2, 3, batchSeriesIdx + 2)
        plt.cla()
        plt.axis([0, backpropagationLength, 0, 2])
        left_offset = range(backpropagationLength)
        plt.bar(left_offset, batchX[batchSeriesIdx, :], width=1, color="blue")
        plt.bar(left_offset, batchY[batchSeriesIdx, :] * 0.5, width=1, color="red")
        plt.bar(left_offset, singleOutputSeries * 0.3, width=1, color="green")

    plt.draw()
    plt.pause(0.0001)

In [6]:
from tqdm import tqdm_notebook
from pywt import wavedec, waverec

"""
run the graph
"""
with tf.Session() as sess:
    writer = tf.summary.FileWriter("./logs/SAE_test8", graph=tf.get_default_graph())
    writer.add_graph(sess.graph)
    sess.run(tf.global_variables_initializer())
    #plt.ion()
    #plt.figure()
    #plt.show()
    loss_list = []

    for epoch_idx in range(numEpochs):
        x,y = generateData()  
        _current_cell_state = np.zeros((batchSize, stateSize))
        _current_hidden_state = np.zeros((batchSize, stateSize))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * backpropagationLength
            end_idx = start_idx + backpropagationLength

            batchX = x[:,start_idx:end_idx]            
            batchY = y[:,start_idx:end_idx]
            
            coeffs = wavedec(batchX, 'haar', level=2)
            reconstruct_X = waverec(coeffs, 'haar')
            
            #_, SAE_cost1 = sess.run([optmzr1, cost1], feed_dict={batchXHolder: reconstruct_X, batchYHolder: batchY})
            #_, SAE_cost2 = sess.run([optmzr2, cost2], feed_dict={batchXHolder: reconstruct_X, batchYHolder: batchY})
            #_, SAE_cost3 = sess.run([optmzr3, cost3], feed_dict={batchXHolder: reconstruct_X, batchYHolder: batchY})
            #_, SAE_cost4 = sess.run([optmzr4, cost4], feed_dict={batchXHolder: reconstruct_X, batchYHolder: batchY}) 

            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, currentState, predictions_series],
                feed_dict={
                    batchXHolder:reconstruct_X,
                    batchYHolder:batchY,
                    cellState: _current_cell_state,
                    hiddenState: _current_hidden_state
                })

            _current_cell_state, _current_hidden_state = _current_state

            loss_list.append(_total_loss)

            # fix the cost summary later
            tf.summary.scalar(name="totalloss", tensor=_total_loss)

            print("Step",batch_idx, "Loss", _total_loss)
            #plot(loss_list, _predictions_series, batchX, batchY)

#plt.ioff()
#plt.show()

x : (5, 50000)
y : (5, 50000)
batchSize :  250
x_reshape : (250, 1000)
y_reshape : (250, 1000)
New data, epoch 0
Step 0 Loss 2.3004253
Step 1 Loss 1.7458103
Step 2 Loss 1.2235825
Step 3 Loss 0.68826276


ValueError: Level value of 2 is too high.  Maximum allowed is 0.